# MAT5014 - Economic research - Suicide rates
*authors: BRAYE Valérien, CHHAY Ly An, ROLLAND Obed* <br>
*language: Python 3* <br>
*dataset: [Suicide Rates Overview 1985 to 2016](https://www.kaggle.com/datasets/russellyates88/suicide-rates-overview-1985-to-2016)*

## Imports and preliminary code
The following cell defines usefull global variables to configure this notebook.
- `DATAPATH`: change this to the path to the directory where the datasets are stored
- `SAVEFIGS`: weither to save the figures to disk or not. Saved figures will not have titles to be included in the report
- `FIGPATH`: path where the figures will be saved if `SAVEFIGS` is set to `True`

In [1]:
DATAPATH = "./data/"
SAVEFIGS = False
FIGPATH = "./report/figures/"

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings("ignore")

The following cell defines a publication-ready `pyplot` theme, taken from [github](https://github.com/matplotlib/matplotlib/issues/19028).

In [ ]:
plot_settings = {'ytick.labelsize': 16,
                 'xtick.labelsize': 16,
                 'font.size': 22,
                 'figure.figsize': (10, 5),
                 'axes.titlesize': 22,
                 'axes.labelsize': 18,
                 'lines.linewidth': 2,
                 'lines.markersize': 3,
                 'legend.fontsize': 11,
                 'mathtext.fontset': 'stix',
                 'font.family': 'STIXGeneral'}
plt.style.use(plot_settings)

The cell below defines usefull functions.

In [ ]:
def print_list(l: list, title: str = None, bullet: str = "•", max_items: int = None, randomize: bool = False, indent_level: int = 0) -> None:
    """
    Prints a list nicely. Handles sublists. Tupples will be displayed as items: ('e1', 'e2', ...).
    Parameters:
        • l: list to print
        • title: title of the list
        • bullet: bullet point to use
        • max_items: maximum number of items to print
        • randomize: pick random elements or not
        • indent_level: indentation level for sub-lists
    """
    if title:
        print(f"{indent_level * '  '}{title}:")
        indent_level += 1
    final_list_to_print = l.copy()
    if randomize:
        random.shuffle(final_list_to_print)
    if max_items:
        final_list_to_print = l[:max_items]
    for item in final_list_to_print:
        if isinstance(item, list):
            print_list(item, bullet=bullet, indent_level=indent_level+1)
        elif isinstance(item, tuple):
            print(f"{indent_level * '  '}{bullet} {str(item)}")
        else:
            print(f"{indent_level * '  '}{bullet} {item}")
    if len(final_list_to_print) != len(l):
        print(f"{indent_level * '  '}...")

## Exploratory Data Analysis
### Presentation of the dataset

In [ ]:
suicides = pd.read_csv(DATAPATH + "master.csv")

In [ ]:
print(f"Dataset dimensions: {suicides.shape[0]} × {suicides.shape[1]}")

In [ ]:
suicides.sample(5, random_state=42)

In [ ]:
suicides.sex.describe()

In [ ]:
print_list(suicides.columns, "Variables in dataset")

The dataset we will be using for this project has almost 30000 records of suicides that happened from 1985 to 2016, in 101 different countries all around the world. For each record 12 variables are given:
- `country`: the country of origin of the person
- `year`: year of the suicide
- `sex`: sex of the person (`"male"` or `"female"`)
- `age`: how old the person was, given as an interval
- `suicide_no`: number of suicides in the same country, sex, year and age group as the person 
- `population`: population size of the country, sex and age category of the person
- `suicides/100k pop`: number of suicides per 100000 people in the country
- `country-year`: concatenation of the country and the year
- `HDI for year`: human developement index of the year
- ` gdp_for_year ($)`: gross domestic product of the year, in dollars
- `gdp_per_capita ($)`: GDP of the year, per person, in dollar
- `generation`: generation of the person

### Data preparation
In this section, we try to exclude outliers, remove `NaN`s and more generaly make the data easier to work with. We will also add the continent based on the country. <br>
First of all, some of the variable names used in the original dataset do not respect snake case, so we will rename them for ease of use.

In [ ]:
columns_rename_map = {
    "suicides/100k pop": "suicides_rate",
    "country-year": "country_year",
    "HDI for year": "hdi",
    " gdp_for_year ($) ": "gdp",
    "gdp_per_capita ($)": "gdp_per_capita",
}

suicides.rename(columns_rename_map, axis=1, inplace=True);

We can also remove useless columns. Since they are redundant with the rest of the data, we will remove variables `country_year` and `generation`.

In [ ]:
suicides.drop(["country_year", "generation"], axis=1, inplace=True)

To make it easier to manipulate the `age` variable, we will transform it according to the table below:
| Original `age` value | New `age` value |
| -------------------- | --------------- |
| `"5-14 years"`       | `0`             |
| `"15-24 years"`      | `1`             |
| `"25-34 years"`      | `2`             |
| `"35-54 years"`      | `3`             |
| `"55-74 years"`      | `4`             |
| `"75+ years"`        | `5`             |

Unfortunatly, not all age categories are of the same size: the last two are 20 years long while the first ones are 10 years long. It is necessary to keep this information in mind later in the study. To compensate for this issue, we will add two variables `age_min` and `age_max`, respectivly the minimum and maximum possible for the considered person. <br>
We will still keep the old `age` variable but rename it to `age_category`.

In [ ]:
suicides["age_category"] = suicides.age

age_map = {
    "5-14 years": 0,
    "15-24 years": 1,
    "25-34 years": 2,
    "35-54 years": 3,
    "55-74 years": 4,
    "75+ years": 5
}

suicides.age = suicides.age.map(age_map)

suicides["age_min"] = suicides.age.map({
    0: 5,
    1: 15,
    2: 25,
    3: 35,
    4: 55,
    5: 75
})
suicides["age_max"] = suicides.age.map({
    0: 14,
    1: 24,
    2: 34,
    3: 54,
    4: 74,
    5: 94 # chosen to make the category 20 years long
})

Note that `gdp` is an object and not a float64 or an int. This is because it uses the american notation, separating 3 digit groups by commas. For instance, `2156624900` is represented by `2,156,624,900`. We will convert them to int in order to be able to manipulate them. 

In [ ]:
suicides.gdp = suicides.gdp.apply(lambda x: int(x.replace(",", "")))

Next, we need to check for missing data.

In [ ]:
for variable in suicides.columns:
    try:
        print(f"{suicides[variable].value_counts(normalize=True, dropna=False)[np.nan]*100:.1f}% of values are missing for variable {variable}.")
    except KeyError:
        print(f"No nan value for variable {variable}.")

There is no missing data except for the `hdi` variable, where more that two thirds of the values are missing. We could either:
- drop the observations with a missing value, which is not acceptable given the proportion of missing values.
- drop the `hdi` variable, but we suspect that it might be higly correlated to the number of suicides so it would be beneficial to keep it.
- fill as much as possible the missing values. This is the option we will take.

For each missing value, we will use the average `hdi` corresponding to the country. If there is not such record, we simply drop the observation.

In [ ]:
def find_hdi(df: pd.DataFrame, country: str) -> int:
    """
    Return the average HDI of the country.
    If no record is found, returns -1.

    Args:
        df (pd.DataFrame): data
        country (str)
        year (int)

    Returns:
        int: HDI of country for year
    """
    records = df[df.country == country].dropna()
    if not records.empty:
        return records.hdi.mean()
    return -1

def fill_hdi_values(df: pd.DataFrame) -> int:
    """
    Fills missing values of the "hdi" variable of the provided dataframe.
    Missing values are filled using the average of existing values from the same country.

    Args:
        df (pd.DataFrame): data.

    Returns:
        int: number of missing values that were replaced.
    """
    memo = {} # stores, for each encountered country pair, the associated HDI
    replaced = 0
    for i, row in df.iterrows():
        if np.isnan(row["hdi"]):
            approx_hdi = -1
            if row["country"] in memo:
                approx_hdi = memo[row["country"]]
            else:
                approx_hdi = find_hdi(df, row["country"])
                memo[row["country"]] = approx_hdi
            if approx_hdi != -1:
                df.hdi[i] = approx_hdi
                replaced += 1
    return replaced

missing_hdi = suicides.hdi.value_counts(dropna=False)[np.nan]
filled_hdi = fill_hdi_values(suicides)
print(f"Filled {filled_hdi} ({filled_hdi/missing_hdi*100:.1f}%) of missing HDI values. About {suicides.hdi.value_counts(normalize=True, dropna=False)[np.nan]*100:.1f}% values are still missing.")
suicides.dropna(inplace=True)
print("All missing values have been droped.")

With this technique, we were able to go from 69% of missing values to about only 5%, that we droped.

### Data visualization
Now that data preparation is done, we can explore it.
First of all, it would be great knowing the represented countries and how many years of records are provided for each of them.

In [ ]:
years_count = suicides.groupby("country").year.nunique().sort_values(ascending=True)
plt.figure(figsize=(10, 22))
sns.barplot(years_count, orient="h")
plt.xlabel("Number of years recorded")
plt.title("Number of years recorded by country");

Countries with the most records have about 30 years of data in the dataframe (so the whole time span of the study), but some other countries have only a few years recorded. However, most countries have at least 20 years recorded. <br>
Let us now study the average suicide rate year.

In [ ]:
sns.lineplot(suicides.groupby("year").suicides_rate.mean())
plt.xlabel("year")
plt.ylabel("suicides per 100k people")
plt.title("Suicide rate evolution");

Suicide rate went from about 12 suicides per 100k people in 1985 to a peak in 1995 at more than 15. This might seem strange at first glance but could actually be explained by the high level of mental distress an economical instability of the time ([see the events of 1995](https://en.wikipedia.org/wiki/1995#Events)). <br>
We can also study the number of suicides depending on some variables about the individuals such as sex or age category.

In [ ]:
age_count = suicides.groupby("age").suicides_rate.mean()
sns.barplot(age_count)
plt.xticks(range(len(age_map)), labels=list(age_map.keys()))
plt.xlabel("Age category")
plt.ylabel("Suicide rate")
plt.title("Suicides between 1985 and 2016");

Most suicides occur when people are between 35 and 54 years, which could be explained by their work and the fact that they start getting older.

In [ ]:
sex_count = suicides.groupby("sex").suicides_no.sum()
plt.pie(sex_count.values, labels=sex_count.index, autopct="%1.1f%%")
plt.title("Suicides between 1985 and 2016");

Most suicides are commited by men. In fact, most of the attempts are done by females, but they more rarely lead to death. Men tend to chose more violent and "effective" ways of commiting suicide and are less prone to seeking help ([source](https://en.wikipedia.org/wiki/Gender_differences_in_suicide)). <br>
Let us now focus on GPD and HDI.

In [ ]:
suicides.gdp_per_capita

In [ ]:
sns.histplot(suicides.gdp_per_capita, kde=True, binwidth=500, ec=None)
plt.title("GPD per capita distribution")
plt.xlabel("GDP per capita ($)")
plt.ylabel("count");

Whithout surprise, most countries have a low GPD per capita and only few of them have a high one.

In [ ]:
plt.figure(figsize=(3, 5))
sns.boxplot(suicides.hdi)
plt.title("HDI distribution")
plt.ylabel("HDI")

As we can seen, the HDI is contained between 0.5 and 1 and the average HDI is about 0.8. Let us try to corellate the GDP and the HDI to the suicide rate.

In [ ]:
hdi_resolution = 50
suicides["hdi_category"] = suicides.hdi.map(lambda x: round(x*hdi_resolution)/hdi_resolution)
sns.lineplot(suicides.groupby("hdi_category").suicides_rate.mean())
plt.title("Suicides depending on HDI")
plt.xlabel("HDI")
plt.ylabel("Suicide rate");

Apparently, there is no linear relationship between the HDI and the suicide rate, as we could have expected. In fact, it even seems that the higher the HDI is, the higher the suicide rate tends to be, except a very high suicide rate for a HDI at about 0.6.

In [ ]:
gdp_per_capita_resolution = .001
suicides["gdp_per_capita_category"] = suicides.gdp_per_capita.map(lambda x: round(x*gdp_per_capita_resolution)/gdp_per_capita_resolution)
sns.lineplot(suicides.groupby("gdp_per_capita_category").suicides_rate.mean())
plt.title("Suicides depending on GDP per capita")
plt.xlabel("GDP per capita ($)")
plt.ylabel("Suicide rate");

There is no clear relationship between the GDP per capita and the average suicide rate.

In [ ]:
gdp_resolution = 0.000000000005
suicides["gdp_category"] = suicides.gdp.map(lambda x: round(x*gdp_resolution)/gdp_resolution)
sns.lineplot(suicides.groupby("gdp_category").suicides_rate.mean())
plt.title("Suicides depending on GDP per capita")
plt.xlabel("GDP ($)")
plt.ylabel("Suicide rate");

On the other hand, GDP seems to have a direct link with the suicide rate in the country. Very low GDP generaly lead to quite low suicide rate. Medium GDP countries record the highest suicide rates. And the suicide rates of high GDP countries are relatively average and stable. 